In [1]:
import tensorlayer as tl
import tensorflow as tf
import rawpy

import model
import utils

from PIL import Image
import numpy as np
import glob
import importlib
from PIL import Image

## Set config

In [2]:
import config as _config
importlib.reload(_config)
from config import config, log_config

###====================== HYPER-PARAMETERS ===========================###
## Adam
batch_size = config.TRAIN.batch_size
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1
## initialize G
n_epoch_init = config.TRAIN.n_epoch_init
## adversarial learning (SRGAN)
n_epoch = config.TRAIN.n_epoch
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every
training_dir = config.TRAIN.training_dir
pretrain_checkpoint = config.TRAIN.pretrain_checkpoint
train_data_dir = config.TRAIN.train_data_dir
dark_model_dir = config.TRAIN.dark_model_dir

crop_num = config.TRAIN.crop_num
sample_img_size = config.TRAIN.sample_img_size
sample_lst = config.TRAIN.sample_lst

tl.files.exists_or_mkdir(training_dir)
tl.files.exists_or_mkdir(pretrain_checkpoint)

[TL] [!] training_dir exists ...
[TL] [!] checkpoint exists ...


True

## Define Model

In [3]:
import model as _model
importlib.reload(_model)
import utils as _utils
importlib.reload(_utils)
import dark_utils as _dark_utils
importlib.reload(_dark_utils)

from model import *
from utils import *
from dark_utils import *

In [4]:
#change

# feed_forward generator
rgb_96_input = tf.placeholder('float32', [None, 96, 96, 3], name='rgb_96_to_SRGAN_generator')
net_g = SRGAN_g(rgb_96_input, is_train=True, reuse=False)
rgb_384_output = dark_network((net_g.outputs + 1)/2, reuse = False)
# label
raw_384 = tf.placeholder('float32', [None, 192, 192, 4], name='raw_384_label')
rgb_384_label = dark_network(raw_384, reuse = True)


# sample_generator
rgb_96_sample = tf.placeholder('float32', [None, None, None, 3], name='rgb_96_sample_to_SRGAN_g_test')
net_g_test = SRGAN_g(rgb_96_sample, is_train=False, reuse=True)
rgb_384_sample = dark_network((net_g_test.outputs + 1)/2, reuse = True)

raw_384_sample = tf.placeholder('float32', [None, None, None, 4], name='raw_384_sample')
rgb_384_sample_from_raw = dark_network(raw_384_sample, reuse = True)

# discriminator
net_d, logits_real = SRGAN_d(rgb_384_label, is_train=True, reuse=False)
_, logits_fake = SRGAN_d(rgb_384_output, is_train=True, reuse=True)

# VGG
rgb_224_label = tf.image.resize_images(
        rgb_384_label,
        size=[224, 224],
        method=0,
        align_corners=False)

rgb_224_output = tf.image.resize_images(
        rgb_384_output,
        size=[224, 224],
        method=0,
        align_corners=False)
    
net_vgg, vgg_target_emb = Vgg19_simple_api(rgb_224_label, reuse=False)
_, vgg_predict_emb = Vgg19_simple_api(rgb_224_output, reuse=True)

net_g.print_params(False)
# net_g.print_layers()
net_d.print_params(False)
# net_d.print_layers()

## test inference


[TL] InputLayer  SRGAN_g/in: (?, 96, 96, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/

[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/add3: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1_new: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1_new: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2_new: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] Conv2d SRGAN_g/out_new: n_filter: 4 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh
[TL] InputLayer  SRGAN_g/in: (?, ?, ?, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 

[TL] BatchNormLayer SRGAN_g/n64s1/b2/14: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/14: size: (?, ?, ?, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/15: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/15: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/15: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/15: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/15: size: (?, ?, ?, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/add3: size: (?, ?, ?, 64) fn: add
[TL] Conv2d SRGAN_g/n2

[TL] Conv2d VGG19/conv4_4: n_filter: 512 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d VGG19/pool4: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d VGG19/conv5_1: n_filter: 512 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv5_2: n_filter: 512 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv5_3: n_filter: 512 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv5_4: n_filter: 512 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d VGG19/pool5: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] FlattenLayer VGG19/flatten: 25088
[TL] DenseLayer  VGG19/fc6: 4096 relu
[TL] DenseLayer  VGG19/fc7: 4096 relu
[TL] DenseLayer  VGG19/fc8: 1000 No Activation
build model finished: 2.158691s
build model started
[TL] InputLayer  VGG19/input: (?, 224, 224, 3)
[TL] Conv2d VGG19/conv1_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[T

[TL]   param  66: SRGAN_g/n64s1/b1/6/moving_variance:0 (64,)              float32_ref
[TL]   param  67: SRGAN_g/n64s1/c2/6/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param  68: SRGAN_g/n64s1/b2/6/beta:0 (64,)              float32_ref
[TL]   param  69: SRGAN_g/n64s1/b2/6/gamma:0 (64,)              float32_ref
[TL]   param  70: SRGAN_g/n64s1/b2/6/moving_mean:0 (64,)              float32_ref
[TL]   param  71: SRGAN_g/n64s1/b2/6/moving_variance:0 (64,)              float32_ref
[TL]   param  72: SRGAN_g/n64s1/c1/7/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param  73: SRGAN_g/n64s1/b1/7/beta:0 (64,)              float32_ref
[TL]   param  74: SRGAN_g/n64s1/b1/7/gamma:0 (64,)              float32_ref
[TL]   param  75: SRGAN_g/n64s1/b1/7/moving_mean:0 (64,)              float32_ref
[TL]   param  76: SRGAN_g/n64s1/b1/7/moving_variance:0 (64,)              float32_ref
[TL]   param  77: SRGAN_g/n64s1/c2/7/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param  78: SRGAN_g/n64s1/b2/7/beta:0 (

[TL]   param 169: SRGAN_g/n256s1/2_new/kernel:0 (3, 3, 64, 256)    float32_ref
[TL]   param 170: SRGAN_g/n256s1/2_new/bias:0 (256,)             float32_ref
[TL]   param 171: SRGAN_g/out_new/kernel:0 (1, 1, 256, 4)     float32_ref
[TL]   param 172: SRGAN_g/out_new/bias:0 (4,)               float32_ref
[TL]   num of params: 1523204
[TL]   param   0: SRGAN_d/h0/c/kernel:0 (4, 4, 3, 64)      float32_ref
[TL]   param   1: SRGAN_d/h0/c/bias:0  (64,)              float32_ref
[TL]   param   2: SRGAN_d/h1/c/kernel:0 (4, 4, 64, 128)    float32_ref
[TL]   param   3: SRGAN_d/h1/bn/beta:0 (128,)             float32_ref
[TL]   param   4: SRGAN_d/h1/bn/gamma:0 (128,)             float32_ref
[TL]   param   5: SRGAN_d/h1/bn/moving_mean:0 (128,)             float32_ref
[TL]   param   6: SRGAN_d/h1/bn/moving_variance:0 (128,)             float32_ref
[TL]   param   7: SRGAN_d/h2/c/kernel:0 (4, 4, 128, 256)    float32_ref
[TL]   param   8: SRGAN_d/h2/bn/beta:0 (256,)             float32_ref
[TL]   param   

## Define training op

In [5]:
d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real), name='d1')
d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake), name='d2')
d_loss = d_loss1 + d_loss2

g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake), name='g')
mse_loss = tl.cost.mean_squared_error(rgb_384_output, rgb_384_label, is_mean=True)
vgg_loss = 2e-6 * tl.cost.mean_squared_error(vgg_predict_emb.outputs, vgg_target_emb.outputs, is_mean=True)

g_loss = mse_loss + vgg_loss + g_gan_loss
# g_loss = mse_loss + g_gan_loss

g_vars = tl.layers.get_variables_with_name('SRGAN_g', True, True)
d_vars = tl.layers.get_variables_with_name('SRGAN_d', True, True)

with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(lr_init, trainable=False)
## Pretrain
g_optim_init = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(mse_loss, var_list=g_vars)
## SRGAN
g_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_loss, var_list=g_vars)
d_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_loss, var_list=d_vars)

[TL]   [*] geting variables with SRGAN_g
[TL]   got   0: SRGAN_g/n64s1/c/kernel:0   (3, 3, 3, 64)
[TL]   got   1: SRGAN_g/n64s1/c/bias:0   (64,)
[TL]   got   2: SRGAN_g/n64s1/c1/0/kernel:0   (3, 3, 64, 64)
[TL]   got   3: SRGAN_g/n64s1/b1/0/beta:0   (64,)
[TL]   got   4: SRGAN_g/n64s1/b1/0/gamma:0   (64,)
[TL]   got   5: SRGAN_g/n64s1/c2/0/kernel:0   (3, 3, 64, 64)
[TL]   got   6: SRGAN_g/n64s1/b2/0/beta:0   (64,)
[TL]   got   7: SRGAN_g/n64s1/b2/0/gamma:0   (64,)
[TL]   got   8: SRGAN_g/n64s1/c1/1/kernel:0   (3, 3, 64, 64)
[TL]   got   9: SRGAN_g/n64s1/b1/1/beta:0   (64,)
[TL]   got  10: SRGAN_g/n64s1/b1/1/gamma:0   (64,)
[TL]   got  11: SRGAN_g/n64s1/c2/1/kernel:0   (3, 3, 64, 64)
[TL]   got  12: SRGAN_g/n64s1/b2/1/beta:0   (64,)
[TL]   got  13: SRGAN_g/n64s1/b2/1/gamma:0   (64,)
[TL]   got  14: SRGAN_g/n64s1/c1/2/kernel:0   (3, 3, 64, 64)
[TL]   got  15: SRGAN_g/n64s1/b1/2/beta:0   (64,)
[TL]   got  16: SRGAN_g/n64s1/b1/2/gamma:0   (64,)
[TL]   got  17: SRGAN_g/n64s1/c2/2/kernel:0  

## restore model

In [6]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                        log_device_placement=False))

In [7]:
tl.layers.initialize_global_variables(sess)
if tl.files.load_and_assign_npz(sess=sess, name=training_dir + '/g_srgan.npz', network=net_g) is None:
    load_pretrain_model(sess=sess, npz_file=pretrain_checkpoint + '/g_srgan.npz', network=net_g)
tl.files.load_and_assign_npz(sess=sess, name=training_dir + '/d_srgan.npz', network=net_d)

[TL] WARNING: Function: `tensorlayer.layers.utils.initialize_global_variables` (in file: /home/hwa125/.local/lib/python3.6/site-packages/tensorlayer/layers/utils.py) is deprecated and will be removed after 2018-09-30.
Instructions for updating: This API is deprecated in favor of `tf.global_variables_initializer`

[TL] ERROR: file training_dir/g_srgan.npz doesn't exist.
[TL] [*] Load checkpoint/g_srgan.npz SUCCESS!
[TL] ERROR: file training_dir/d_srgan.npz doesn't exist.


In [8]:
var_dict = dark_model_var_dict(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='DARK'))
var_dict_1 = {
    'Variable': var_dict['1/Variable'],
    'Variable_1': var_dict['1/Variable_1'],
    'Variable_2': var_dict['1/Variable_2'],
    'Variable_3': var_dict['1/Variable_3']
}
var_dict_2 = {
    'Variable': var_dict['2/Variable'],
    'Variable_1': var_dict['2/Variable_1'],
    'Variable_2': var_dict['2/Variable_2'],
    'Variable_3': var_dict['2/Variable_3']
}
var_dict_3 = {
    'Variable': var_dict['3/Variable'],
    'Variable_1': var_dict['3/Variable_1'],
    'Variable_2': var_dict['3/Variable_2'],
    'Variable_3': var_dict['3/Variable_3']
}
for i in range(1,4): 
    var_dict.pop('%i/Variable' % i)
    var_dict.pop('%i/Variable_1' % i)
    var_dict.pop('%i/Variable_2' % i)
    var_dict.pop('%i/Variable_3' % i)

In [9]:
#load the pre-train dark model
saver = tf.train.Saver(var_dict)
saver_1 = tf.train.Saver(var_dict_1)
saver_2 = tf.train.Saver(var_dict_2)
saver_3 = tf.train.Saver(var_dict_3)

ckpt = tf.train.get_checkpoint_state(dark_model_dir)
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)
    saver_1.restore(sess, ckpt.model_checkpoint_path)
    saver_2.restore(sess, ckpt.model_checkpoint_path)
    saver_3.restore(sess, ckpt.model_checkpoint_path)

loaded dark_model/Sony/model.ckpt
INFO:tensorflow:Restoring parameters from dark_model/Sony/model.ckpt
INFO:tensorflow:Restoring parameters from dark_model/Sony/model.ckpt
INFO:tensorflow:Restoring parameters from dark_model/Sony/model.ckpt
INFO:tensorflow:Restoring parameters from dark_model/Sony/model.ckpt


In [10]:
vgg19_npy_path = "vgg19.npy"
if not os.path.isfile(vgg19_npy_path):
    print("Please download vgg19.npz from : https://github.com/machrisaa/tensorflow-vgg")
    exit()
npz = np.load(vgg19_npy_path, encoding='latin1').item()

params = []
for val in sorted(npz.items()):
    W = np.asarray(val[1][0])
    b = np.asarray(val[1][1])
    print("  Loading %s: %s, %s" % (val[0], W.shape, b.shape))
    params.extend([W, b])
tl.files.assign_params(sess, params, net_vgg)

  Loading conv1_1: (3, 3, 3, 64), (64,)
  Loading conv1_2: (3, 3, 64, 64), (64,)
  Loading conv2_1: (3, 3, 64, 128), (128,)
  Loading conv2_2: (3, 3, 128, 128), (128,)
  Loading conv3_1: (3, 3, 128, 256), (256,)
  Loading conv3_2: (3, 3, 256, 256), (256,)
  Loading conv3_3: (3, 3, 256, 256), (256,)
  Loading conv3_4: (3, 3, 256, 256), (256,)
  Loading conv4_1: (3, 3, 256, 512), (512,)
  Loading conv4_2: (3, 3, 512, 512), (512,)
  Loading conv4_3: (3, 3, 512, 512), (512,)
  Loading conv4_4: (3, 3, 512, 512), (512,)
  Loading conv5_1: (3, 3, 512, 512), (512,)
  Loading conv5_2: (3, 3, 512, 512), (512,)
  Loading conv5_3: (3, 3, 512, 512), (512,)
  Loading conv5_4: (3, 3, 512, 512), (512,)
  Loading fc6: (25088, 4096), (4096,)
  Loading fc7: (4096, 4096), (4096,)
  Loading fc8: (4096, 1000), (1000,)


ResourceExhaustedError: OOM when allocating tensor of shape [25088,4096] and type float
	 [[{{node Assign_199/value}} = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [25088,4096] values: [0.000740604708 -9.02350512e-05 0.000419194781...]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'Assign_199/value', defined at:
  File "/opt/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.6/site-packages/tornado/pla

## get a sub set of sample, test for training

In [31]:
p = Pool(3)

# train_data_list = sorted(tl.files.load_file_list('test_data', regx = '^0.*.ARW', printable = False))
train_data_list = sorted(tl.files.load_file_list('test_data', regx = '.*.ARW', printable = False))

In [32]:
# sample_index = list(range(0, 320, 80))
sample_lst = [0]
# sample_lst = range(4)
ni = int(np.sqrt(len(sample_lst)))

sample_file_name = [train_data_list[i] for i in sample_lst]

In [33]:
# rgb_sample, raw_sample = get_inputs_labels(p = p,
#                                           file_dir = train_data_dir,
#                                           raw_file_list = sample_file_name,
#                                           crop_num = None,
#                                           crop_size = 1000)
rgb_sample, raw_sample = get_inputs_labels(p = p,
                                          file_dir = 'test_data',
                                          raw_file_list = sample_file_name,
                                          crop_num = None,
                                          crop_size = 500)

In [34]:
# Save the sample input rgb
rgb_sample = ((rgb_sample+1)/2 * 255)
rgb_sample_out_filename = training_dir + os.path.sep + 'rgb_sample.png'
if not os.path.exists(rgb_sample_out_filename):
    tl.vis.save_images(rgb_sample, [ni, ni], rgb_sample_out_filename)

In [35]:
# Save the sample label raw

label_raw_sample = sess.run(rgb_384_sample_from_raw, feed_dict={raw_384_sample: raw_sample})
label_raw_sample = np.minimum(np.maximum(label_raw_sample, 0), 1)
label_raw_sample = (label_raw_sample * 255)
label_raw_sample_out_file_name = training_dir + os.path.sep + 'label_raw_sample.png'
if not os.path.exists(label_raw_sample_out_file_name):
    tl.vis.save_images(label_raw_sample, [ni, ni], label_raw_sample_out_file_name)

## Training
### initialize G

In [16]:
sess.run(tf.assign(lr_v, lr_init))
print(" ** fixed learning rate: %f (for init G)" % lr_init)

 ** fixed learning rate: 0.000100 (for init G)


In [17]:
batch_size = 2
crop_num = 1

In [18]:
for epoch in range(0, 40 + 1):
    epoch_time = time.time()
    total_mse_loss, n_iter = 0, 0

    for idx in range(0, len(train_data_list), batch_size):
        step_time = time.time()
        
        batch_file_name = train_data_list[idx: idx+batch_size]
#         inputs_rgbs, label_raws = get_inputs_labels(p, train_data_dir, batch_file_name, crop_num)
        inputs_rgbs, label_raws = get_inputs_labels(p, 'test_data', batch_file_name, crop_num)
        
        ## update G
        errM, _ = sess.run([mse_loss, g_optim_init], {t_image: inputs_rgbs, t_target_image: label_raws})
        print("Epoch [%2d/%2d] %4d time: %4.4fs, mse: %.8f " % (epoch, n_epoch_init, n_iter, time.time() - step_time, errM))
        total_mse_loss += errM
        n_iter += 1
    log = "[*] Epoch: [%2d/%2d] time: %4.4fs, mse: %.8f" % (epoch, n_epoch_init, time.time() - epoch_time, total_mse_loss / n_iter)
    print(log)

    # quick evaluation on train set
    if (epoch != 0) and (epoch % 10 == 0):
        sample_out = sess.run(net_g_test.outputs, {t_image_sample: rgb_sample})
        sample_out = pack_raw_matrix(sample_out) * 200
        
        train_raw_sample = sess.run(out_image, feed_dict={t_raw_for_dark: sample_out})
        train_raw_sample = np.minimum(np.maximum(train_raw_sample, 0), 1)
        train_raw_sample = (train_raw_sample * 255).astype(np.uint8)
        train_raw_sample_out_file_name = training_dir + os.path.sep + '/train_raw_sample_%d.png' % epoch
        
        print("[*] save images")
        tl.vis.save_images(train_raw_sample, [ni, ni], train_raw_sample_out_file_name)
            
    ## save model
    if (epoch != 0) and (epoch % 10 == 0):
        tl.files.save_npz(net_g.all_params, name=training_dir + '/g_srgan_init.npz', sess=sess)

Epoch [ 0/100]    0 time: 4.2848s, mse: 0.82553345 
Epoch [ 0/100]    1 time: 1.3810s, mse: 0.79789329 
Epoch [ 0/100]    2 time: 1.2712s, mse: 0.72667342 
Epoch [ 0/100]    3 time: 1.2810s, mse: 0.82550436 
[*] Epoch: [ 0/100] time: 8.2182s, mse: 0.79390113
Epoch [ 1/100]    0 time: 1.3650s, mse: 0.77747792 
Epoch [ 1/100]    1 time: 1.4443s, mse: 0.74258214 
Epoch [ 1/100]    2 time: 1.3296s, mse: 0.69772798 
Epoch [ 1/100]    3 time: 1.4641s, mse: 0.49750084 
[*] Epoch: [ 1/100] time: 5.6032s, mse: 0.67882222
Epoch [ 2/100]    0 time: 1.2875s, mse: 0.68143964 
Epoch [ 2/100]    1 time: 1.3845s, mse: 0.54008365 
Epoch [ 2/100]    2 time: 1.3346s, mse: 0.56234610 
Epoch [ 2/100]    3 time: 1.2997s, mse: 0.46491694 
[*] Epoch: [ 2/100] time: 5.3065s, mse: 0.56219658
Epoch [ 3/100]    0 time: 1.3828s, mse: 0.62870270 
Epoch [ 3/100]    1 time: 1.4353s, mse: 0.54932535 
Epoch [ 3/100]    2 time: 1.2011s, mse: 0.49835545 
Epoch [ 3/100]    3 time: 1.3952s, mse: 0.55458212 
[*] Epoch: [ 3/

[*] save images
[TL] [*] Saving TL params into training_dir/g_srgan_init.npz
[TL] [*] Saved
Epoch [31/100]    0 time: 1.3886s, mse: 0.00462554 
Epoch [31/100]    1 time: 1.3737s, mse: 0.01797284 
Epoch [31/100]    2 time: 1.3927s, mse: 0.00368195 
Epoch [31/100]    3 time: 1.3109s, mse: 0.00322281 
[*] Epoch: [31/100] time: 5.4662s, mse: 0.00737579
Epoch [32/100]    0 time: 1.2444s, mse: 0.00576146 
Epoch [32/100]    1 time: 1.3361s, mse: 0.00367709 
Epoch [32/100]    2 time: 1.3201s, mse: 0.02259646 
Epoch [32/100]    3 time: 1.2731s, mse: 0.00392294 
[*] Epoch: [32/100] time: 5.1739s, mse: 0.00898949
Epoch [33/100]    0 time: 1.3282s, mse: 0.00499573 
Epoch [33/100]    1 time: 1.3182s, mse: 0.00477029 
Epoch [33/100]    2 time: 1.3526s, mse: 0.00404269 
Epoch [33/100]    3 time: 1.2430s, mse: 0.00418914 
[*] Epoch: [33/100] time: 5.2423s, mse: 0.00449946
Epoch [34/100]    0 time: 1.3730s, mse: 0.00359722 
Epoch [34/100]    1 time: 1.3653s, mse: 0.00328161 
Epoch [34/100]    2 time: 1

In [22]:
for epoch in range(0, 40 + 1):
    ## update learning rate
    if epoch != 0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay**(epoch // decay_every)
        sess.run(tf.assign(lr_v, lr_init * new_lr_decay))
        log = " ** new learning rate: %f (for GAN)" % (lr_init * new_lr_decay)
        print(log)
    elif epoch == 0:
        sess.run(tf.assign(lr_v, lr_init))
        log = " ** init lr: %f  decay_every_init: %d, lr_decay: %f (for GAN)" % (lr_init, decay_every, lr_decay)
        print(log)

    epoch_time = time.time()
    total_d_loss, total_g_loss, n_iter = 0, 0, 0

    for idx in range(0, len(train_data_list), batch_size):
        step_time = time.time()
        batch_file_name = train_data_list[idx: idx+batch_size]
#         inputs_rgbs, label_raws = get_inputs_labels(p, train_data_dir, batch_file_name, crop_num)
        inputs_rgbs, label_raws = get_inputs_labels(p, 'test_data', batch_file_name, crop_num)
        ## update D
        errD, _ = sess.run([d_loss, d_optim], {t_image: inputs_rgbs, t_target_image: label_raws})
        ## update G
        errG, errM, errA, _ = sess.run([g_loss, mse_loss, g_gan_loss, g_optim],
                                             {t_image: inputs_rgbs, t_target_image: label_raws})
        print("Epoch [%2d/%2d] %4d time: %4.4fs, d_loss: %.8f g_loss: %.8f (mse: %.6f adv: %.6f)" %
              (epoch, n_epoch, n_iter, time.time() - step_time, errD, errG, errM, errA))
        total_d_loss += errD
        total_g_loss += errG
        n_iter += 1

    log = "[*] Epoch: [%2d/%2d] time: %4.4fs, d_loss: %.8f g_loss: %.8f" % (epoch, n_epoch, time.time() - epoch_time, total_d_loss / n_iter,
                                                                            total_g_loss / n_iter)
    print(log)

    # quick evaluation on train set
    if (epoch != 0) and (epoch % 10 == 0):
        sample_out = sess.run(net_g_test.outputs, {t_image_sample: rgb_sample})
        sample_out = pack_raw_matrix(sample_out) * 200
        
        train_raw_sample = sess.run(out_image, feed_dict={t_raw_for_dark: sample_out})
        train_raw_sample = np.minimum(np.maximum(train_raw_sample, 0), 1)
        train_raw_sample = (train_raw_sample * 255).astype(np.uint8)
        train_raw_sample_out_file_name = training_dir + os.path.sep + '/train_raw_sample_%d(GAN).png' % epoch
        
        print("[*] save images")
        tl.vis.save_images(train_raw_sample, [ni, ni], train_raw_sample_out_file_name)

    ## save model
    if (epoch != 0) and (epoch % 10 == 0):
        tl.files.save_npz(net_g.all_params, name=training_dir + '/g_srgan.npz', sess=sess)
        tl.files.save_npz(net_d.all_params, name=training_dir + '/d_srgan.npz', sess=sess)

 ** init lr: 0.000100  decay_every_init: 1000, lr_decay: 0.100000 (for GAN)
Epoch [ 0/2000]    0 time: 1.9502s, d_loss: 2.52115321 g_loss: 0.00374823 (mse: 0.002150 adv: 0.001598)
Epoch [ 0/2000]    1 time: 1.9604s, d_loss: 0.38484010 g_loss: 0.02049297 (mse: 0.014041 adv: 0.006452)
Epoch [ 0/2000]    2 time: 1.8715s, d_loss: 2.44681311 g_loss: 0.00923260 (mse: 0.007457 adv: 0.001775)
Epoch [ 0/2000]    3 time: 1.9933s, d_loss: 3.55068922 g_loss: 0.00211774 (mse: 0.000801 adv: 0.001317)
[*] Epoch: [ 0/2000] time: 7.7757s, d_loss: 2.22587391 g_loss: 0.00889789
Epoch [ 1/2000]    0 time: 1.9835s, d_loss: 0.14413087 g_loss: 0.00623906 (mse: 0.001515 adv: 0.004724)
Epoch [ 1/2000]    1 time: 1.9573s, d_loss: 2.16561866 g_loss: 0.00857060 (mse: 0.004541 adv: 0.004029)
Epoch [ 1/2000]    2 time: 1.9668s, d_loss: 7.51290512 g_loss: 0.00624027 (mse: 0.002106 adv: 0.004135)
Epoch [ 1/2000]    3 time: 1.8213s, d_loss: 3.38742065 g_loss: 0.03051618 (mse: 0.029201 adv: 0.001315)
[*] Epoch: [ 1/200

Epoch [16/2000]    2 time: 1.9155s, d_loss: 3.12672067 g_loss: 0.00359301 (mse: 0.002507 adv: 0.001086)
Epoch [16/2000]    3 time: 1.9720s, d_loss: 2.18370509 g_loss: 0.00401797 (mse: 0.000908 adv: 0.003110)
[*] Epoch: [16/2000] time: 7.8164s, d_loss: 2.07963455 g_loss: 0.00381888
Epoch [17/2000]    0 time: 2.0329s, d_loss: 2.66017056 g_loss: 0.00195472 (mse: 0.000615 adv: 0.001340)
Epoch [17/2000]    1 time: 1.9494s, d_loss: 2.42755389 g_loss: 0.00844134 (mse: 0.006270 adv: 0.002171)
Epoch [17/2000]    2 time: 1.9348s, d_loss: 3.04890609 g_loss: 0.00426535 (mse: 0.003159 adv: 0.001107)
Epoch [17/2000]    3 time: 1.8577s, d_loss: 1.60033560 g_loss: 0.00281314 (mse: 0.000753 adv: 0.002060)
[*] Epoch: [17/2000] time: 7.7751s, d_loss: 2.43424153 g_loss: 0.00436864
Epoch [18/2000]    0 time: 1.8836s, d_loss: 5.64814138 g_loss: 0.00666251 (mse: 0.002869 adv: 0.003794)
Epoch [18/2000]    1 time: 1.9535s, d_loss: 0.91201079 g_loss: 0.01069581 (mse: 0.009197 adv: 0.001499)
Epoch [18/2000]    2

Epoch [32/2000]    3 time: 2.0660s, d_loss: 12.50316620 g_loss: 0.01567148 (mse: 0.012927 adv: 0.002744)
[*] Epoch: [32/2000] time: 7.8320s, d_loss: 4.94691247 g_loss: 0.00809228
Epoch [33/2000]    0 time: 1.9699s, d_loss: 1.11265266 g_loss: 0.00474504 (mse: 0.001754 adv: 0.002991)
Epoch [33/2000]    1 time: 1.9608s, d_loss: 3.37703562 g_loss: 0.00502800 (mse: 0.001673 adv: 0.003355)
Epoch [33/2000]    2 time: 1.9302s, d_loss: 9.54619312 g_loss: 0.01261944 (mse: 0.004697 adv: 0.007923)
Epoch [33/2000]    3 time: 1.9678s, d_loss: 3.58529615 g_loss: 0.07749112 (mse: 0.070908 adv: 0.006584)
[*] Epoch: [33/2000] time: 7.8289s, d_loss: 4.40529439 g_loss: 0.02497090
Epoch [34/2000]    0 time: 2.0040s, d_loss: 12.63529682 g_loss: 0.00972597 (mse: 0.001050 adv: 0.008676)
Epoch [34/2000]    1 time: 1.9176s, d_loss: 0.90438032 g_loss: 0.01266002 (mse: 0.006358 adv: 0.006302)
Epoch [34/2000]    2 time: 1.8936s, d_loss: 2.38615632 g_loss: 0.01082266 (mse: 0.005838 adv: 0.004984)
Epoch [34/2000]   

In [ ]:
sample_out.max()

In [ ]:
rgb_sample.shape

In [ ]:
tl.vis.save_images(rgb_sample, [1, 1], 'test_png')

In [ ]:
rgb_sample.shape